In [1]:
import numpy as np
import pandas as pd

import datawig

import os
import time
import gc


In [ ]:
# train_X = pd.read_csv('./data/train.csv')
# train_y = train_X['SalePrice']
# train_X = train_X.drop(['SalePrice', 'Id'], axis=1)
# test_X = pd.read_csv('./data/test.csv')

In [17]:
train_X = pd.read_csv('./data/train.csv')
train_y = train_X['SalePrice']
train_X = pd.read_csv('./imputed_train.csv')
test_X = pd.read_csv('./imputed_test.csv')
test_X = test_X.drop('Id', axis=1)

In [18]:
cols = train_X.columns
cat = []
num = []
ctd = []
imps = []
for c in cols:
    if train_X[c].dtype == 'object' or test_X[c].dtype == 'object':
        cat.append(c)
    else:
        num.append(c)
        
    if np.sum(train_X[c].isna())/len(train_X) > 0.9 or np.sum(test_X[c].isna())/len(test_X)>0.9:
        ctd.append(c)
        
    if (np.sum(test_X[c].isna())>0 or np.sum(train_X[c].isna())>0) and c not in ctd:
        imps.append(c)

In [19]:
train_X = train_X.drop(ctd, axis=1)
test_X = test_X.drop(ctd, axis=1)


In [42]:
print(train_X.shape, test_X.shape)

(1460, 76) (1459, 76)


In [5]:
for c in imps:
    imputer = datawig.SimpleImputer(
    input_columns=train_X.columns, # column(s) containing information about the column we want to impute
    output_column=c, # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

    #Fit an imputer model on the train data
    imputer.fit(train_df=train_X)

    #Impute missing values and return original dataframe with predictions
    imputed = imputer.predict(train_X)
    
    new_c = c+'_imputed'
    train_X[c] = imputed[new_c]
    
    

In [43]:
X = train_X.copy()
X2 = test_X.copy()

In [27]:
for c in train_X.columns:
    k = np.sum(train_X[c].isna())
    if c in cat and k>0:
        print('cat', c)
    elif c in num and k>0:
        print('num', c)

In [44]:
for c in cat:
    temp = pd.get_dummies(X2[c], prefix=c)
    X2 = pd.concat([X2, temp], axis=1)
    X2 = X2.drop(c, axis=1)
    
for n in num:
    X2[n] = (X2[n] - np.mean(X2[n]))/np.std(X2[n])

In [45]:
for c in cat:
    temp = pd.get_dummies(X[c], prefix=c)
    X = pd.concat([X, temp], axis=1)
    X = X.drop(c, axis=1)
    
for n in num:
    X[n] = (X[n] - np.mean(X[n]))/np.std(X[n])

In [46]:
print(X.shape, X2.shape)

(1460, 274) (1459, 256)


In [47]:
for c in X.columns:
    if c not in X2.columns:
        X2[c] = np.ones(len(X2))

In [48]:
for c in X2.columns:
    if c not in X.columns:
        X[c] = np.ones(len(X))
        print(c)

Functional_Maj2


In [49]:
print(X.shape, X2.shape)

(1460, 275) (1459, 275)


In [50]:
X.to_csv('./train_X.csv', index=False)
X2.to_csv('./test_X.csv', index=False)

In [56]:
train_y.to_csv('./train_y.csv', index=False, header=True)